In [1]:
import pandas as pd, numpy as np

In [2]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')

In [3]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [4]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]

In [5]:
dc=dc.set_index('uid')
dm=dm.set_index('uid')

In [6]:
dmi=dm.loc[dc.index.dropna()]

In [7]:
df=dmi.merge(dc).drop_duplicates()

Working df ready. Now infer location.

In [8]:
locs=df[['category','icategory','tags','title','content']]

In [9]:
locs

,category,icategory,tags,title,content
0,aktualis,csik,"['Kovászna megye', 'medve']","Újabb medvét gázoltak el, ezúttal Kovászna meg...",Elgázolt vasárnap este egy gépkocsi egy medvét...
1,aktualis,agerpres,"['Homoródalmás', 'medve']","Érdeklődik a medve a füstölt sonka iránt, de a...",Több háziállatot is megölt Homoródalmáson egy ...
2,aktualis,csik,"['medve', 'Tusnádfürdő']",Bocsával együtt tört be az udvarra az anyamedve,Ismét medvéhez riasztották keddre virradóan a ...
3,aktualis,csik,"['Hargita megye', 'medve']",Még a kormányválság is a garázdálkodó medvékne...,Már a veszélyes medvék kilövésére sem kapnak e...
4,aktualis,csik,"['medve', 'Udvarhelyszék']",Nagykadácson és Felsőboldogfalván is medve mia...,Medve miatt riasztották csütörtökre virradóra ...
5,aktualis,csik,['Zetelaka'],"Nem sokat ér a medvebiztos kuka, ha mellé tesz...","Még nem tudni, hogy bevált-e a nyáron Sikaszób..."
6,aktualis,vasarhely,"['Közlekedés', 'Környezetvédelem', 'Marosvásár...",Már most kellene számolni a medvékkel a Marosv...,Tíz százalékkal növelné a Marosvásárhely–Jászv...
7,vilag,csik,"['Belföld', 'medve']",Medgyes utcáiról zavarták el a medvét,"A csendőrök, rendőrök és erdészek visszazavart..."
8,vilag,csik,['Belföld'],Lépcsőházban járt a medve,Egy fogarasi tömbház lépcsőházában látott medv...
9,aktualis,udvarhely,"['medve', 'Székelyudvarhely']",Sebzett medvét üldöznek a vadászok Székelyudva...,Kétbocsos anyamedvét ütöttek el hétfőn este Sz...


In [10]:
import json

In [11]:
lak=json.loads(open('geo/huncoords.json','r').read())
megyek=json.loads(open('geo/megyek.json','r').read())

In [12]:
laks=[i for i in megyek.keys() if ((megyek[i]!='Más megye')and(i in lak.keys()))]

In [13]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [14]:
#look for perfect matches
decoded_locs={}
min_loc_len=5
for i in locs.index:
    if '[' in locs.loc[i]['tags']:
        k=locs.loc[i]['tags'].replace("'",'"')
        for j in json.loads(k):
            if j in laks:
                if i not in decoded_locs:decoded_locs[i]=[]
                decoded_locs[i].append(j)
                print('1',k,j)
    for j in locs.loc[i]['title'].split():
        if j in laks:
            if i not in decoded_locs:decoded_locs[i]=[]
            decoded_locs[i].append(j)
            print('2',j)
    for j in locs.loc[i]['content'].split():
        if j in laks:
            if i not in decoded_locs:decoded_locs[i]=[]
            decoded_locs[i].append(j)
            print('3',j)
    for j in locs.loc[i]['title'].split():
        for k in laks:
            if ((len(j)>min_loc_len) and (len(k)>min_loc_len)):
                if levenshteinDistance(j,k)<3:
                    if i not in decoded_locs:decoded_locs[i]=[]
                    decoded_locs[i].append(k)
                    print('4',j,k)
    for j in locs.loc[i]['content'].split():
        for k in laks:
            if ((len(j)>min_loc_len) and (len(k)>min_loc_len)):
                if levenshteinDistance(j,k)<3:
                    if i not in decoded_locs:decoded_locs[i]=[]
                    decoded_locs[i].append(k)
                    print('5',j,k)

2 Kovászna
3 Kovászna
3 Ozsdola
4 Kovászna Kovászna
5 Kovászna Kovászna
5 Ozsdola Ozsdola
1 ["Homoródalmás", "medve"] Homoródalmás
5 Homoródalmáson Homoródalmás
1 ["medve", "Tusnádfürdő"] Tusnádfürdő
5 Tusnádfürdőn. Tusnádfürdő
4 Felsőboldogfalván Felsőboldogfalva
1 ["Zetelaka"] Zetelaka
1 ["Közlekedés", "Környezetvédelem", "Marosvásárhely"] Marosvásárhely
3 Brassó
5 fogarasi Fogaras
5 Brassó Brassó
1 ["medve", "Székelyudvarhely"] Székelyudvarhely
2 Székelyudvarhely
3 Székelyudvarhely
3 Felsőboldogfalva
4 Székelyudvarhely Székelyudvarhely
5 Székelyudvarhely Székelyudvarhely
5 Felsőboldogfalva Felsőboldogfalva
5 keresi Keresd
1 ["Székelyudvarhely"] Székelyudvarhely
4 Székelyudvarhelyen Székelyudvarhely
5 székelyudvarhelyi Székelyudvarhely
3 Torja
5 Daragus Dragus


In [15]:
predicted_locs=[]
for i in range(len(locs.index)):
    if locs.index[i] in decoded_locs: 
        dummy=list(set(decoded_locs[locs.index[i]]))
        if len(dummy)==1:
            dummy=dummy[0]
        else:
            dummy=repr(list(set(decoded_locs[locs.index[i]])))
        predicted_locs.append(dummy)
    else: 
        predicted_locs.append('')

In [16]:
locs['predicted_locs']=predicted_locs
locs['actual_locs']=predicted_locs

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
lm=locs[['title','content','predicted_locs','actual_locs']]

In [18]:
lm.to_excel('data/szekelyhon_locs.xlsx')
lm.to_excel('data/szekelyhon_locs_curated.xlsx')

Open `data/szekelyhon_locs_curated` and manually verify predicted locations. Amend `actual_locs` where necessary.